# **Topic Modeling and Search with Top2Vec :** 
# **S.E.A Aquarium**

# **1. Import and Setup**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/My Drive/nlp')

## **a) To install Top2Vec library and pre-trained BERT sentence transformer options:**

In [3]:
# !pip install top2vec[sentence_transformers]
!pip install pynndescent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 26.3 MB/s 
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=c5519288d62ddf58b184a244e2b194d56d1b22ca380a55b320b472e0c9be7972
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built pynndescent



## **b) Import Libraries**

In [4]:
from top2vec import Top2Vec 

import pandas as pd
import numpy as np

# **2. Import Dataset**

In [5]:
df = pd.read_excel("/content/gdrive/MyDrive/data/clean/formatted_reviews.xlsx")
df.sample(4)

,date,source,attraction,reviews,rating
925,2019.0,google_reviews,uss,sekali kali boleh lah ajak anak liburan ke sini,10.0
4538,2019.0,traveloka,wings_of_time,salah satu pertunjukan terbaik di singapura,9.0
3494,2019.0,traveloka,sea_aquarium,anakanak sangat menikmati sea aquarium juga be...,10.0
3068,2019.0,traveloka,uss,beli tiket di traveloka sangat menyenangkan ha...,10.0


In [6]:
df['date'] = df['date'].astype(int)

In [7]:
aqua_df = df[df['attraction'] == 'sea_aquarium']
aqua_df = aqua_df.sort_values(by=['date','rating'],ascending=[False,False])
# resetting index
aqua_df = aqua_df.reset_index(drop=True)
aqua_df.tail(4)

,date,source,attraction,reviews,rating
640,2017,tripadvisor,sea_aquarium,edukatif anak saya senang sekali berada di sin...,8.0
641,2017,tripadvisor,sea_aquarium,fish everywhere saya kesana dengan 1 anak dan ...,8.0
642,2017,tripadvisor,sea_aquarium,aquarium super besar dengan shark melimpah mas...,8.0
643,2017,tripadvisor,sea_aquarium,kurang berkesan entah mengapa saya merasa seaw...,6.0


In [8]:
aqua_docs = aqua_df.loc[:, "reviews"].astype(str).values.tolist()
aqua_docs[:7]

['banyak hewan menarik yang bisa dilihat tempatnya luas dan instagenic',
 'sangat indah pemandangan lautnya berseta biotalautnya',
 'bagus lumayan lengkap biota lautnya terawat dengan baik bersih',
 'sangat membantu anak2 dlm mengenal jenis2 ikan dan anak2 sangat senang sekali melihat aquarium raksasa',
 'mantap deh bawa anakanak kesini ga nyesel',
 'masa allah so beautiful place thank you for sea aquarium singapura',
 'mantapkerenbanyak model ikan']

# **3. Create and Train N-Gram model (Bi-Gram and Tri-Gram)**
* Top2Vec uses Gensim **simple_preprocess** to do tokenization. 
* To create Trigrams and Bigrams, pass a customized tokenizer into Top2Vec tokenizer parameter. 
* Use Gensim to train this N-gram model. Documents passed to the Phrases function to inspect for N-grams text corpus.

In [10]:
import gensim
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import strip_tags

# Build the bigram and trigram models 
sentence_stream = [doc.split(" ") for doc in aqua_docs]
bigram = Phrases(sentence_stream, min_count=5, threshold=5, delimiter=b' ')
trigram = Phrases(bigram[sentence_stream], threshold=5, delimiter=b' ')

bigram_phraser = Phraser(bigram)
trigram_phraser = Phraser(trigram)

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
# Create Bi-Grams
def bi_gram(doc):
    sentence_stream = simple_preprocess(strip_tags(str(doc)), deacc=True)
    return bigram_phraser[sentence_stream] 


# Create Bi-Grams and Tri-Grams
def tri_gram(doc):
    sentence_stream = simple_preprocess(strip_tags(str(doc)), deacc=True)
    gen_bigram = bigram_phraser[sentence_stream]
    gen_trigram = trigram_phraser[bigram_phraser[sentence_stream]]
    return gen_trigram

# **4. Train Top2Vec Model**

### **Parameters**:
* **Documents**: Input corpus, should be a list of strings.
* **Min_count**: (Optional, default 50) Ignores all words with total frequency lower than this. For smaller corpora a smaller min_count will be necessary.
* **Embedding_model** (string or callable) – The valid string options are: doc2vec , universal-sentence-encoder , universal-sentence-encoder-multilingual,distiluse-base-multilingual-cased , all-MiniLM-L6-v2 , paraphrase-multilingual-MiniLM-L12-v2 
* **tokenizer** (callable (Optional, default None)) – Override the default tokenization method. If None then gensim.utils.simple_preprocess will be used. ***Tokenizer must take a document and return a list of tokens***.


In [12]:
# FORM UNI-GRAMS, BI-GRAMS WITH BI-GRAM MODEL 

topicMODEL_bi = Top2Vec(documents= aqua_docs, min_count=5, embedding_model = 'distiluse-base-multilingual-cased', tokenizer= bi_gram)  

2022-06-26 05:36:17,555 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-26 05:36:17,775 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model


Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/528 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

2022-06-26 05:36:55,624 - top2vec - INFO - Creating joint document/word embedding
2022-06-26 05:37:35,168 - top2vec - INFO - Creating lower dimension embedding of documents
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-06-26 05:37:47,032 - top2vec - INFO - Finding dense areas of documents
2022-06-26 05:37:47,053 - top2vec - INFO - Finding topics


In [13]:
# FORM UNI-GRAMS, BI-GRAMS, TRI-GRAMS WITH TRI-GRAM MODEL 
 
topicMODEL_tri = Top2Vec(documents= aqua_docs, min_count=5, embedding_model = 'distiluse-base-multilingual-cased', tokenizer = tri_gram)  

2022-06-26 05:37:47,074 - top2vec - INFO - Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-06-26 05:37:47,281 - top2vec - INFO - Downloading distiluse-base-multilingual-cased model
2022-06-26 05:37:49,473 - top2vec - INFO - Creating joint document/word embedding
2022-06-26 05:38:26,828 - top2vec - INFO - Creating lower dimension embedding of documents
2022-06-26 05:38:31,953 - top2vec - INFO - Finding dense areas of documents
2022-06-26 05:38:31,970 - top2vec - INFO - Finding topics


In [14]:
# SAVE MODELS FOR FUTURE USE

topicMODEL_bi.save("/content/gdrive/MyDrive/data/models/bigram_aqua_june26")

topicMODEL_tri.save("/content/gdrive/MyDrive/data/models/trigram_aqua_june26")

In [ ]:
# LOAD MODEL FROM SAVED MODELS

topicMODEL_bi = Top2Vec.load("/content/gdrive/MyDrive/data/models/bigram_aqua_june23")

topicMODEL_tri = Top2Vec.load("/content/gdrive/MyDrive/data/models/trigram_aqua_june23")

# **5. Perform Topic Reduction** 

* Reduce the number of topics discovered by Top2Vec.
* The most representative topics of the corpus will be found, by iteratively merging each smallest topic to the most similar topic until num_topics is reached. 
* Get the hierarchy of reduced topics. The mapping of each original topic to the reduced topics is returned.

In [ ]:
# BI-GRAM MODEL TOPIC REDUCTION
topicMODEL_bi.hierarchical_topic_reduction(num_topics=10) 

topicMODEL_bi.get_topic_hierarchy()

In [ ]:
# TRI-GRAM MODEL TOPIC REDUCTION 
topicMODEL_tri.hierarchical_topic_reduction(num_topics=10) 

topicMODEL_tri.get_topic_hierarchy()

# **6. Explore Discovered Topics**

## **6a) Get Number of Topics**
This will return the number of topics that Top2Vec has found in the data.

In [15]:
# Show the Total Number of Topics 

print(f"Total Number of Topics generated (Topic Reduction Applied): \n")
print(f"Bi-gram Model : {topicMODEL_bi.get_num_topics(reduced=False)}")
print(f"Tri-gram Model : {topicMODEL_tri.get_num_topics(reduced=False)}")

Total Number of Topics generated (Topic Reduction Applied): 

Bi-gram Model : 3
Tri-gram Model : 3


## **6b) Get Topic Sizes**
This will return the number of documents most similar to each topic. Topics are in decreasing order of size.

Returns:
* topic_sizes: The number of documents most similar to each topic.
* topic_nums: The unique index of every topic will be returned.

### **6b.1. Bi-Gram and Tri-Gram Model Topic Sizes**

In [16]:
print("BI-GRAM MODEL where N = 1 and 2")
print("================================\n")
topic_sizes_bi, topic_nums_bi = topicMODEL_bi.get_topic_sizes(reduced=False) 

print(f"Unique index numbers of every topic: {topic_nums_bi} ")
print()
print(f"Number of documents for each unique topic: {topic_sizes_bi} \n")

print("TRI-GRAM MODEL where N = 1, 2 and 3")
print("====================================\n")
topic_sizes_tri, topic_nums_tri = topicMODEL_tri.get_topic_sizes(reduced=False) 
print(f"Unique index numbers of every topic: {topic_nums_tri} ")
print()
print(f"Number of documents for each unique topic: {topic_sizes_tri} ") 

BI-GRAM MODEL where N = 1 and 2

Unique index numbers of every topic: [0 1 2] 

Number of documents for each unique topic: [355 168 121] 

TRI-GRAM MODEL where N = 1, 2 and 3

Unique index numbers of every topic: [0 1 2] 

Number of documents for each unique topic: [354 168 122] 


## **6c) Get Topics**
This will return the topics in decreasing size.

Returns:

* topic_words: For each topic the top 50 words are returned, in decreasing order of semantic similarity to topic.

* word_scores: For each topic the cosine similarity scores (in decreasing order) of the top 50 words to the topic are returned.

* topic_nums: The unique index of every topic will be returned.

### **c.1.  BI-GRAM Model Topics**

In [17]:
print(f"TOPICS FOR BI-GRAM MODEL: \n")
topic_words_bi, word_scores_bi, topic_nums_bi = topicMODEL_bi.get_topics(topicMODEL_bi.get_num_topics(reduced=False))

for count,(words_bi, scores_bi, num_bi) in enumerate(zip(topic_words_bi, word_scores_bi, topic_nums_bi)):
    print(f"{count+1}. Topic {num_bi+1}") 
    print(f"Topic Words : \n{words_bi} \n")
    # print(f"Cosine Similarity :\n{scores_bi}\n")

TOPICS FOR BI-GRAM MODEL: 

1. Topic 1
Topic Words : 
['sea aquarium' 'aquarium raksasa' 'aquarium terbesar' 'aquariumnya'
 'aquarium' 'biota laut' 'jenis ikan' 'fish' 'hewan laut' 'akuarium'
 'macam ikan' 'ikan nya' 'koleksi ikan' 'ikannya' 'shark' 'ikan hiu'
 'ikan pari' 'ocean' 'laut yang' 'bawah laut' 'lautnya' 'sea' 'sea world'
 'ke sea' 'ikan' 'ikanikan' 'lumba' 'lumbalumba' 'bintang laut' 'laut'
 'spesies' 'kolam' 'biota' 'kapal' 'hewan' 'binatang'
 'sangat menyenangkan' 'sangat indah' 'tempatnya bagus' 'ubur' 'ribet'
 'singapore' 'ke singapore' 'banyak jenis' 'great' 'sangat senang'
 'menakjubkan' 'indah' 'tempatnya luas' 'suka sekali'] 

2. Topic 2
Topic Words : 
['tempatnya bagus' 'sangat menyenangkan' 'bagus banget' 'sangat senang'
 'menyenangkan' 'fun' 'bagus untuk' 'membawa keluarga' 'sangat indah'
 'liburan keluarga' 'menikmati' 'bersama keluarga' 'suka sekali' 'indah'
 'great' 'good' 'menakjubkan' 'bawa anakanak' 'bagus' 'awesome' 'excited'
 'luar biasa' 'bahagia' 'anak 

### **c.2. Tri-Gram Model Topics**

In [18]:
print(f"TOPICS FOR TRI-GRAM MODEL : \n")
topic_words_tri, word_scores_tri, topic_nums_tri = topicMODEL_tri.get_topics(topicMODEL_tri.get_num_topics(reduced=False))

for count, (words_tri, scores_tri, num_tri) in enumerate(zip(topic_words_tri, word_scores_tri, topic_nums_tri)):
    print(f"{count+1}. Topic {num_tri+1}") 
    print(f"Topic Words : \n{words_tri}\n")
    # print(f"Cosine Similarity :\n{scores_tri}\n")

TOPICS FOR TRI-GRAM MODEL : 

1. Topic 1
Topic Words : 
['di sea aquarium' 'ke sea aquarium' 'sea aquarium' 'aquarium raksasa'
 'aquarium terbesar' 'berbagai macam ikan' 'aquariumnya' 'aquarium'
 'biota laut' 'aquarium lain' 'jenis ikan' 'fish' 'hewan laut' 'akuarium'
 'macam ikan' 'koleksi ikan' 'ikan nya' 'ikannya' 'shark' 'ikan hiu'
 'ikan pari' 'ocean' 'laut yang' 'bawah laut' 'lautnya' 'sea'
 'di bawah laut' 'sea world' 'ke sea' 'ikan' 'ikanikan' 'lumba'
 'lumbalumba' 'bintang laut' 'laut' 'spesies' 'kolam' 'biota' 'kapal'
 'hewan' 'binatang' 'sangat menyenangkan' 'sangat indah' 'tempatnya bagus'
 'ubur' 'ribet' 'suka banget' 'sangat bagus' 'di singapore' 'singapore']

2. Topic 2
Topic Words : 
['tempatnya bagus' 'sangat menyenangkan' 'bagus banget' 'sangat senang'
 'menyenangkan' 'bagus untuk' 'sangat bagus' 'fun' 'membawa keluarga'
 'sangat indah' 'liburan keluarga' 'menikmati' 'suka banget'
 'bersama keluarga' 'suka sekali' 'indah' 'sangat menarik' 'great'
 'bawa anakanak' 'goo

## **6d) Search Documents by Topic**
We are going to search by topic; for example Topic 3. 

Returns:

* documents:  The documents in a list, the most similar are first.
* doc_scores:  Semantic similarity of document to topic. The cosine similarity of the document and topic vector.
* doc_ids:  Unique ids of documents. If ids were not given, the index of document in the original corpus.

For each of the returned documents we are going to print its content, score and document number.

### **d.1. Search Documents By Topic Number Using Bi-Gram Model**

In [19]:
# SEARCH DOCUMENTS BY TOPIC NUMBER USING BI-GRAM MODEL 
topic_num_bi = 2
documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= 5, reduced=False) 

print(f"TOPIC NUMBER: {topic_num_bi}\n")
for count, (doc_bi, docscore_bi, doc_id_bi) in enumerate(zip(documents_bi, document_scores_bi, document_ids_bi)):
    print(f"{count}. Document: {doc_id_bi},  Score: {docscore_bi}")
    print("-----------")
    print(doc_bi)
    print("-----------\n")

TOPIC NUMBER: 2

0. Document: 438,  Score: 0.6679750680923462
-----------
beli tiket lewat traveloka tidak perlu antri untuk tukar tiket yang sudah dibeli langsung bisa scan dan masuk mempermudah sekali
-----------

1. Document: 324,  Score: 0.6674647331237793
-----------
sangat mudah membeli tiket masuk melalui klook tidak perlu mengantri lagi untuk membeli tiket masuk hanya dengan menunjukkan konfirmasi dari aplikasi dan scan barcode saja untuk masuk ke wahana
-----------

2. Document: 348,  Score: 0.6630101203918457
-----------
easy to usetinggal scan barcode tidak perlu antri beli tiket
-----------

3. Document: 243,  Score: 0.6510140895843506
-----------
penggunaan tiket sangat mudah hanya scan barcode yang dikirim oleh pihak traveloka tanpa perlu ditukar tiket fisik sangat terbantu sekali terima kasih traveloka
-----------

4. Document: 387,  Score: 0.6405320763587952
-----------
terima kasih to traveloka lebih memudahkan saya dalam pembelian kupon maupun tiket tanpa antri langsu

### **d.2. Search Documents By Topic Number Using Tri-Gram Model**

In [20]:
# SEARCH DOCUMENTS BY TOPIC NUMBER USING TRI-GRAM MODEL 
topic_num_tri = 1
documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri, num_docs= 5, reduced=False) 
print(f"TOPIC NUMBER: {topic_num_bi}\n")
for count, (doc_tri, docscore_tri, doc_id_tri) in enumerate(zip(documents_tri, document_scores_tri, document_ids_tri)):
    print(f"{count}. Document: {doc_id_tri},  Score: {docscore_tri}")
    print("-----------")
    print(doc_tri)
    print("-----------\n")

TOPIC NUMBER: 2

0. Document: 95,  Score: 0.6289916038513184
-----------
nice place to spend your time with family especially for children my daughter very bahagia at this place
-----------

1. Document: 418,  Score: 0.5972644686698914
-----------
its so amazing my kids so bahagia they are so excited
-----------

2. Document: 460,  Score: 0.5825139284133911
-----------
nice place recomended to family trip
-----------

3. Document: 489,  Score: 0.5401506423950195
-----------
bagus dan anakanak sangat menikmati kunjungannya
-----------

4. Document: 466,  Score: 0.5057018399238586
-----------
nice and fun
-----------



# **7) Constructing Dataframe to Collate Information of Key Topics**

## **7a) BI-GRAM MODEL**

In [36]:
# CONSTRUCT DATAFRAME OF BI-GRAM MODEL WITH KEY INFORMATION FOR EXPORT
# initialize data of lists.
bigram_table = {
              'topic number': [num_bi for num_bi in topic_nums_bi],
              'topic words': [words_bi for words_bi in topic_words_bi],
              'cosine similarity metrics': [scores_bi for scores_bi in word_scores_bi],
              'similar documents': [size_bi for size_bi in topic_sizes_bi],
               }
 
# Create DataFrame
bigram_df = pd.DataFrame(bigram_table)
bigram_df['topic number'] = bigram_df['topic number'].apply(lambda x: x + 1)

# print(bigram_df.head(2))
bigram_df.tail(3)

,topic number,topic words,cosine similarity metrics,similar documents
0,1,"[sea aquarium, aquarium raksasa, aquarium terb...","[0.6360315, 0.6063454, 0.5725639, 0.551149, 0....",355
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49217114, 0.4842132, 0.389907, 0.37995502, ...",168
2,3,"[membeli tiket, ticket, scan barcode, sangat m...","[0.41245317, 0.38974592, 0.2832086, 0.21928698...",121


### **7a.1. Build Functions to Expand Column Features in Bi-Gram Dataframe**

In [37]:
def get_document_index(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    document_index = [doc_id_bi for doc_id_bi in document_ids_bi]
    return document_index 

def get_document_scores(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    document_scores = [docscore_bi for docscore_bi in document_scores_bi]
    return document_scores


def get_documents(topic_num_bi, num_doc):
    documents_bi, document_scores_bi, document_ids_bi = topicMODEL_bi.search_documents_by_topic(topic_num= topic_num_bi , num_docs= num_doc, reduced=False)
    actual_document = [doc_bi for doc_bi in documents_bi]
    return actual_document


In [38]:
bigram_df['document index'] = bigram_df['topic number'].apply(lambda x: get_document_index(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df['document scores'] = bigram_df['topic number'].apply(lambda x: get_document_scores(x-1, bigram_df['similar documents'].values[x-1]))

bigram_df['actual reviews'] = bigram_df['topic number'].apply(lambda x: get_documents(x-1, bigram_df['similar documents'].values[x-1]))


# bigram_df = bigram_df.drop(['topic label'], axis = 1)
bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews
0,1,"[sea aquarium, aquarium raksasa, aquarium terb...","[0.6360315, 0.6063454, 0.5725639, 0.551149, 0....",355,"[493, 598, 373, 161, 244, 624, 635, 518, 397, ...","[0.7237828, 0.7027494, 0.6927036, 0.68469584, ...",[sangat menyenangkan aquariumnya sangat bersih...
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49217114, 0.4842132, 0.389907, 0.37995502, ...",168,"[95, 418, 460, 489, 466, 357, 117, 507, 219, 2...","[0.62745357, 0.5968965, 0.58033353, 0.5399852,...",[nice place to spend your time with family esp...
2,3,"[membeli tiket, ticket, scan barcode, sangat m...","[0.41245317, 0.38974592, 0.2832086, 0.21928698...",121,"[438, 324, 348, 243, 387, 327, 345, 540, 230, ...","[0.66797507, 0.66746473, 0.6630101, 0.6510141,...",[beli tiket lewat traveloka tidak perlu antri ...


In [39]:
# for count, (index, review) in enumerate(zip(bigram_df['document index'][28], bigram_df['actual reviews'][28])):
#   print(f"{count}: {index} = {review})")

bigram_df.to_excel(f'/content/gdrive/MyDrive/data/export/bigram_df_aqua.xlsx', index=False)

## **7b) TRI-GRAM MODEL**

In [40]:
# CONSTRUCT DATAFRAME CONTAINING KEY INFORMATION FOR EXPORT
# initialize data of lists.
trigram_table = {
                'topic number': [num_tri for num_tri in topic_nums_tri],
                'topic words': [words_tri for words_tri in topic_words_tri],
                'cosine similarity metrics': [scores_tri for scores_tri in word_scores_tri],
                'similar documents': [size_tri for size_tri in topic_sizes_tri],
                 }

# Create DataFrame
trigram_df = pd.DataFrame(trigram_table)

trigram_df['topic number'] = trigram_df['topic number'].apply(lambda x: x + 1)
trigram_df.tail(3)

,topic number,topic words,cosine similarity metrics,similar documents
0,1,"[di sea aquarium, ke sea aquarium, sea aquariu...","[0.6707168, 0.66935956, 0.6359184, 0.60672903,...",354
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49807966, 0.48428267, 0.39270902, 0.3797741...",168
2,3,"[membeli tiket, ticket, tinggal scan barcode, ...","[0.40689868, 0.38336587, 0.32091275, 0.2931899...",122


### **7b.1. Build Functions to Expand Column Features in Tri-Gram Dataframe**

In [41]:
def get_document_index_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    document_index = [doc_id_tri for doc_id_tri in document_ids_tri]
    return document_index 

def get_document_scores_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    document_scores = [docscore_tri for docscore_tri in document_scores_tri]
    return document_scores


def get_documents_tri(topic_num_tri, num_docs):
    documents_tri, document_scores_tri, document_ids_tri = topicMODEL_tri.search_documents_by_topic(topic_num= topic_num_tri , num_docs= num_docs, reduced=False)
    actual_document = [doc_tri for doc_tri in documents_tri]
    return actual_document


In [42]:
trigram_df['document index'] = trigram_df['topic number'].apply(lambda x: get_document_index_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df['document scores'] = trigram_df['topic number'].apply(lambda x: get_document_scores_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df['actual reviews'] = trigram_df['topic number'].apply(lambda x: get_documents_tri(x-1, trigram_df['similar documents'].values[x-1]))

trigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews
0,1,"[di sea aquarium, ke sea aquarium, sea aquariu...","[0.6707168, 0.66935956, 0.6359184, 0.60672903,...",354,"[493, 598, 373, 161, 244, 624, 635, 518, 397, ...","[0.7239162, 0.7028072, 0.6931868, 0.6847985, 0...",[sangat menyenangkan aquariumnya sangat bersih...
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49807966, 0.48428267, 0.39270902, 0.3797741...",168,"[95, 418, 460, 489, 466, 357, 507, 117, 219, 2...","[0.6289916, 0.59726447, 0.5825139, 0.54015064,...",[nice place to spend your time with family esp...
2,3,"[membeli tiket, ticket, tinggal scan barcode, ...","[0.40689868, 0.38336587, 0.32091275, 0.2931899...",122,"[324, 348, 438, 243, 387, 327, 345, 352, 230, ...","[0.6689373, 0.6654334, 0.6647038, 0.6529148, 0...",[sangat mudah membeli tiket masuk melalui kloo...


In [43]:
# for count, (index, review) in enumerate(zip(trigram_df['document index'][0], trigram_df['actual reviews'][0])):
#   print(f"{count}: {index} = {review})")

trigram_df.to_excel(f'/content/gdrive/MyDrive/data/export/trigram_df_aqua.xlsx', index=False)

# **8. Defining Topic Labels on Reviews** 

## **8a) User-Define Topic Labels** 

### **8a.1:  BI-GRAM Model** 

In [44]:
# Create the dictionary containing the data of the new column
bigram_col_dict = { 'Topic 1': 'features visitors like',                 'Topic 2': 'vistitors sentiments',
                    'Topic 3': 'visitors experiences',           
                  }
                   
# Assign the values of the dictionary as the values of the new column
bigram_df['topic label'] = bigram_col_dict.values()
bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[sea aquarium, aquarium raksasa, aquarium terb...","[0.6360315, 0.6063454, 0.5725639, 0.551149, 0....",355,"[493, 598, 373, 161, 244, 624, 635, 518, 397, ...","[0.7237828, 0.7027494, 0.6927036, 0.68469584, ...",[sangat menyenangkan aquariumnya sangat bersih...,features visitors like
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49217114, 0.4842132, 0.389907, 0.37995502, ...",168,"[95, 418, 460, 489, 466, 357, 117, 507, 219, 2...","[0.62745357, 0.5968965, 0.58033353, 0.5399852,...",[nice place to spend your time with family esp...,vistitors sentiments
2,3,"[membeli tiket, ticket, scan barcode, sangat m...","[0.41245317, 0.38974592, 0.2832086, 0.21928698...",121,"[438, 324, 348, 243, 387, 327, 345, 540, 230, ...","[0.66797507, 0.66746473, 0.6630101, 0.6510141,...",[beli tiket lewat traveloka tidak perlu antri ...,visitors experiences


In [45]:
# CREATE COPY OF ORIGINAL DATAFRAME
aqua_DF_BI = aqua_df.copy()
aqua_DF_BI.tail(3)

,date,source,attraction,reviews,rating
641,2017,tripadvisor,sea_aquarium,fish everywhere saya kesana dengan 1 anak dan ...,8.0
642,2017,tripadvisor,sea_aquarium,aquarium super besar dengan shark melimpah mas...,8.0
643,2017,tripadvisor,sea_aquarium,kurang berkesan entah mengapa saya merasa seaw...,6.0


In [46]:
print([i for i in range(topicMODEL_bi.get_num_topics(reduced=False))])

[0, 1, 2]


In [47]:
# FUNCTION TO CONNECT TOPIC LABELS TO INDEX LIST
def get_indexLIST(i):
      topic = [doc_ind for doc_ind in bigram_df['document index'][i]]
      return topic

In [48]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    aqua_DF_BI.loc[get_indexLIST(i), 'topic label'] = bigram_df['topic label'][i]
    

In [49]:
# FUNCTION TO CONNECT DOCUMENT COSINE SCORE TO INDEX LIST
def get_scores(i):
    score = [doc_score for doc_score in bigram_df['document scores'][i]]
    return score

In [50]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    aqua_DF_BI.loc[get_indexLIST(i), 'cosine score review/topic'] = get_scores(i) 

In [51]:
aqua_DF_BI.reset_index(drop=True)
aqua_DF_BI

,date,source,attraction,reviews,rating,topic label,cosine score review/topic
0,2022,google_reviews,sea_aquarium,banyak hewan menarik yang bisa dilihat tempatn...,10.0,features visitors like,0.316550
1,2022,google_reviews,sea_aquarium,sangat indah pemandangan lautnya berseta biota...,10.0,features visitors like,0.441024
2,2022,google_reviews,sea_aquarium,bagus lumayan lengkap biota lautnya terawat de...,10.0,features visitors like,0.381022
3,2022,google_reviews,sea_aquarium,sangat membantu anak2 dlm mengenal jenis2 ikan...,10.0,features visitors like,0.647345
4,2022,traveloka,sea_aquarium,mantap deh bawa anakanak kesini ga nyesel,10.0,vistitors sentiments,0.210323
...,...,...,...,...,...,...,...
639,2017,tripadvisor,sea_aquarium,aquariumnya bagus ikan nya bermacam macam di t...,8.0,features visitors like,0.609388
640,2017,tripadvisor,sea_aquarium,edukatif anak saya senang sekali berada di sin...,8.0,features visitors like,0.569134
641,2017,tripadvisor,sea_aquarium,fish everywhere saya kesana dengan 1 anak dan ...,8.0,features visitors like,0.602262
642,2017,tripadvisor,sea_aquarium,aquarium super besar dengan shark melimpah mas...,8.0,features visitors like,0.578102


In [53]:
# TO VERIFY TOPIC LABELING IS CORRECT: 
# STEP 1 - GENERATE RANDOM SAMPLE TABLE FROM NEW DATAFRAME 
bigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[sea aquarium, aquarium raksasa, aquarium terb...","[0.6360315, 0.6063454, 0.5725639, 0.551149, 0....",355,"[493, 598, 373, 161, 244, 624, 635, 518, 397, ...","[0.7237828, 0.7027494, 0.6927036, 0.68469584, ...",[sangat menyenangkan aquariumnya sangat bersih...,features visitors like
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49217114, 0.4842132, 0.389907, 0.37995502, ...",168,"[95, 418, 460, 489, 466, 357, 117, 507, 219, 2...","[0.62745357, 0.5968965, 0.58033353, 0.5399852,...",[nice place to spend your time with family esp...,vistitors sentiments
2,3,"[membeli tiket, ticket, scan barcode, sangat m...","[0.41245317, 0.38974592, 0.2832086, 0.21928698...",121,"[438, 324, 348, 243, 387, 327, 345, 540, 230, ...","[0.66797507, 0.66746473, 0.6630101, 0.6510141,...",[beli tiket lewat traveloka tidak perlu antri ...,visitors experiences


In [54]:
# TO VERIFY TOPIC LABELING IS CORRECT: 
# STEP 2 - SELECT INDEXES FROM ORIGINAL DATAFRAME TO COMPARE

aqua_DF_BI.iloc[[493, 95, 438]] 

,date,source,attraction,reviews,rating,topic label,cosine score review/topic
493,2019,traveloka,sea_aquarium,sangat menyenangkan aquariumnya sangat bersih ...,8.0,features visitors like,0.723783
95,2020,traveloka,sea_aquarium,nice place to spend your time with family espe...,9.0,vistitors sentiments,0.627454
438,2019,traveloka,sea_aquarium,beli tiket lewat traveloka tidak perlu antri u...,9.0,visitors experiences,0.667975


In [55]:
aqua_DF_BI.to_excel(f'/content/gdrive/MyDrive/data/export/aqua_DF_BI.xlsx', index=False)

### **8a.2  TRI-GRAM Model** 

In [56]:
# Create the dictionary containing the data of the new column
trigram_col_dict = { 'Topic 1': 'features visitors like',                 'Topic 2': 'vistitors sentiments',
                    'Topic 3': 'visitors experiences',           
                  }
   

# Assign the values of the dictionary as the values of the new column
trigram_df['topic label'] = trigram_col_dict.values()
trigram_df.tail(3)

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[di sea aquarium, ke sea aquarium, sea aquariu...","[0.6707168, 0.66935956, 0.6359184, 0.60672903,...",354,"[493, 598, 373, 161, 244, 624, 635, 518, 397, ...","[0.7239162, 0.7028072, 0.6931868, 0.6847985, 0...",[sangat menyenangkan aquariumnya sangat bersih...,features visitors like
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49807966, 0.48428267, 0.39270902, 0.3797741...",168,"[95, 418, 460, 489, 466, 357, 507, 117, 219, 2...","[0.6289916, 0.59726447, 0.5825139, 0.54015064,...",[nice place to spend your time with family esp...,vistitors sentiments
2,3,"[membeli tiket, ticket, tinggal scan barcode, ...","[0.40689868, 0.38336587, 0.32091275, 0.2931899...",122,"[324, 348, 438, 243, 387, 327, 345, 352, 230, ...","[0.6689373, 0.6654334, 0.6647038, 0.6529148, 0...",[sangat mudah membeli tiket masuk melalui kloo...,visitors experiences


In [57]:
# CREATE COPY OF ORIGINAL DATAFRAME
aqua_DF_TRI = aqua_df.copy()
aqua_DF_TRI.tail(3)

,date,source,attraction,reviews,rating
641,2017,tripadvisor,sea_aquarium,fish everywhere saya kesana dengan 1 anak dan ...,8.0
642,2017,tripadvisor,sea_aquarium,aquarium super besar dengan shark melimpah mas...,8.0
643,2017,tripadvisor,sea_aquarium,kurang berkesan entah mengapa saya merasa seaw...,6.0


In [58]:
print([i for i in range(topicMODEL_tri.get_num_topics(reduced=False))])

[0, 1, 2]


In [61]:
# FUNCTION TO CONNECT TOPIC LABELS TO INDEX LIST
def get_indexLIST(i):
      topic = [doc_ind for doc_ind in trigram_df['document index'][i]]
      return topic

In [62]:
for i in range(topicMODEL_bi.get_num_topics(reduced=False)):
    aqua_DF_TRI.loc[get_indexLIST(i), 'topic label'] = trigram_df['topic label'][i]
    

In [63]:
# FUNCTION TO CONNECT DOCUMENT COSINE SCORE TO INDEX LIST
def get_scores(i):
    score = [doc_score for doc_score in trigram_df['document scores'][i]]
    return score

In [64]:
for i in range(topicMODEL_tri.get_num_topics(reduced=False)):
    aqua_DF_TRI.loc[get_indexLIST(i), 'cosine score review/topic'] = get_scores(i) 

In [65]:
aqua_DF_TRI.reset_index(drop=True)
aqua_DF_TRI

,date,source,attraction,reviews,rating,topic_label,topic label,cosine score review/topic
0,2022,google_reviews,sea_aquarium,banyak hewan menarik yang bisa dilihat tempatn...,10.0,features visitors like,features visitors like,0.316896
1,2022,google_reviews,sea_aquarium,sangat indah pemandangan lautnya berseta biota...,10.0,features visitors like,features visitors like,0.441045
2,2022,google_reviews,sea_aquarium,bagus lumayan lengkap biota lautnya terawat de...,10.0,features visitors like,features visitors like,0.381224
3,2022,google_reviews,sea_aquarium,sangat membantu anak2 dlm mengenal jenis2 ikan...,10.0,features visitors like,features visitors like,0.647516
4,2022,traveloka,sea_aquarium,mantap deh bawa anakanak kesini ga nyesel,10.0,vistitors sentiments,vistitors sentiments,0.212035
...,...,...,...,...,...,...,...,...
639,2017,tripadvisor,sea_aquarium,aquariumnya bagus ikan nya bermacam macam di t...,8.0,features visitors like,features visitors like,0.609626
640,2017,tripadvisor,sea_aquarium,edukatif anak saya senang sekali berada di sin...,8.0,features visitors like,features visitors like,0.569308
641,2017,tripadvisor,sea_aquarium,fish everywhere saya kesana dengan 1 anak dan ...,8.0,features visitors like,features visitors like,0.602052
642,2017,tripadvisor,sea_aquarium,aquarium super besar dengan shark melimpah mas...,8.0,features visitors like,features visitors like,0.578356


In [66]:
# TO VERIFY TOPIC LABELING IS CORRECT: 
# STEP 1 - GENERATE RANDOM SAMPLE TABLE FROM NEW DATAFRAME 
trigram_df

,topic number,topic words,cosine similarity metrics,similar documents,document index,document scores,actual reviews,topic label
0,1,"[di sea aquarium, ke sea aquarium, sea aquariu...","[0.6707168, 0.66935956, 0.6359184, 0.60672903,...",354,"[493, 598, 373, 161, 244, 624, 635, 518, 397, ...","[0.7239162, 0.7028072, 0.6931868, 0.6847985, 0...",[sangat menyenangkan aquariumnya sangat bersih...,features visitors like
1,2,"[tempatnya bagus, sangat menyenangkan, bagus b...","[0.49807966, 0.48428267, 0.39270902, 0.3797741...",168,"[95, 418, 460, 489, 466, 357, 507, 117, 219, 2...","[0.6289916, 0.59726447, 0.5825139, 0.54015064,...",[nice place to spend your time with family esp...,vistitors sentiments
2,3,"[membeli tiket, ticket, tinggal scan barcode, ...","[0.40689868, 0.38336587, 0.32091275, 0.2931899...",122,"[324, 348, 438, 243, 387, 327, 345, 352, 230, ...","[0.6689373, 0.6654334, 0.6647038, 0.6529148, 0...",[sangat mudah membeli tiket masuk melalui kloo...,visitors experiences


In [67]:
# TO VERIFY TOPIC LABELING IS CORRECT: 
# STEP 2 - SELECT INDEXES FROM ORIGINAL DATAFRAME TO COMPARE

aqua_DF_TRI.iloc[[493, 95, 324]] 

,date,source,attraction,reviews,rating,topic_label,topic label,cosine score review/topic
493,2019,traveloka,sea_aquarium,sangat menyenangkan aquariumnya sangat bersih ...,8.0,features visitors like,features visitors like,0.723916
95,2020,traveloka,sea_aquarium,nice place to spend your time with family espe...,9.0,vistitors sentiments,vistitors sentiments,0.628992
324,2019,klook,sea_aquarium,sangat mudah membeli tiket masuk melalui klook...,10.0,visitors experiences,visitors experiences,0.668937


In [69]:
aqua_DF_TRI.to_excel(f'/content/gdrive/MyDrive/data/export/aqua_DF_TRI.xlsx', index=False)

In [70]:
aqua_DF_TRI.isnull().values.any()

False

# **9.  Visualization**

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
uss_topred = pd.read_excel("/content/gdrive/MyDrive/data/export/uss_DF_BIreduced.xlsx")
uss_topred.isnull().values.any()
uss_topred = uss_topred.loc[uss_topred['date'] != 2017]
uss_topred

In [ ]:
sns.color_palette("gist_rainbow")

In [ ]:
sns.set_palette("gist_rainbow")
sns.set_theme(style="ticks", font_scale=1.4)

In [ ]:
plt.figure(figsize=(18,10))

hue_order = [2022, 2021, 2020, 2019, 2018]
sns.histplot(data=uss_topred, y="topic_label", hue="date", hue_order=hue_order, multiple="stack", bins= 50, palette="Set1") 


In [ ]:
sns.color_palette('Set3')
plt.figure(figsize=(25,10))

hue_order = [2018, 2019, 2020,2021, 2022]
sns.histplot(data=uss_topred, x="topic_label", hue="date", hue_order=hue_order, multiple="dodge", bins=100, palette="Set1") 


In [ ]:
plt.figure(figsize=(16,11))

sns.countplot(data=uss_topred, y="topic_label", hue="date", dodge=False)